In [1]:
#!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/MLZoomCamp/HomeWorks/bank+marketing/bank-full.csv")

In [4]:
df = df['age;"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"'].str.split(';', expand=True)

In [5]:
df.columns = ['age', 'job','marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day','month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

In [6]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,"""management""","""married""","""tertiary""","""no""",2143,"""yes""","""no""","""unknown""",5,"""may""",261,1,-1,0,"""unknown""","""no"""
1,44,"""technician""","""single""","""secondary""","""no""",29,"""yes""","""no""","""unknown""",5,"""may""",151,1,-1,0,"""unknown""","""no"""
2,33,"""entrepreneur""","""married""","""secondary""","""no""",2,"""yes""","""yes""","""unknown""",5,"""may""",76,1,-1,0,"""unknown""","""no"""
3,47,"""blue-collar""","""married""","""unknown""","""no""",1506,"""yes""","""no""","""unknown""",5,"""may""",92,1,-1,0,"""unknown""","""no"""
4,33,"""unknown""","""single""","""unknown""","""no""",1,"""no""","""no""","""unknown""",5,"""may""",198,1,-1,0,"""unknown""","""no"""


In [7]:
#df.job.replace('"',"")
df = df.replace('"', '', regex=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  object
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  object
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  object
 10  month      45211 non-null  object
 11  duration   45211 non-null  object
 12  campaign   45211 non-null  object
 13  pdays      45211 non-null  object
 14  previous   45211 non-null  object
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: object(17)
memory usage: 5.9+ MB


In [9]:
new_df = df[['age', 'job','marital', 'education', 'balance', 'housing', 'contact', 'day','month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']]

In [10]:
df = new_df.copy()

In [11]:
df.isnull().sum()

,0
age,0
job,0
marital,0
education,0
balance,0
housing,0
contact,0
day,0
month,0
duration,0


In [12]:
df.education.value_counts()

,count
education,
secondary,23202
tertiary,13301
primary,6851
unknown,1857


In [13]:
df[['age', 'balance' , 'day' , 'duration' , 'pdays', 'previous' ,'campaign']] = df[['age', 'balance' , 'day' , 'duration' , 'pdays', 'previous', 'campaign']].astype(int)

In [14]:
df2 = df[['age', 'balance' , 'day' , 'duration' , 'pdays', 'previous','campaign']]
df2.corr()

,age,balance,day,duration,pdays,previous,campaign
age,1.000000,0.097783,-0.009120,-0.004648,-0.023758,0.001288,0.004760
balance,0.097783,1.000000,0.004503,0.021560,0.003435,0.016674,-0.014578
day,-0.009120,0.004503,1.000000,-0.030206,-0.093044,-0.051710,0.162490
duration,-0.004648,0.021560,-0.030206,1.000000,-0.001565,0.001203,-0.084570
pdays,-0.023758,0.003435,-0.093044,-0.001565,1.000000,0.454820,-0.088628
previous,0.001288,0.016674,-0.051710,0.001203,0.454820,1.000000,-0.032855
campaign,0.004760,-0.014578,0.162490,-0.084570,-0.088628,-0.032855,1.000000


In [15]:
#Target encoding

df.y = (df.y == 'yes').astype(int)

In [16]:
from sklearn.model_selection import train_test_split

df_train_full , df_test = train_test_split(df,test_size=.2 , random_state=42)
df_train , df_val = train_test_split(df_train_full,test_size=.25 , random_state=42)

In [17]:
len(df) *.6 ,len(df_test) ,len(df_val) , len(df_train)

(27126.6, 9043, 9042, 27126)

In [18]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

del df_train['y']
del df_test['y']
del df_val['y']

In [19]:
df_train_full.y.mean()

0.11606945366069453

In [20]:
categorical_cols = df_train.select_dtypes(include=['object']).columns
numerical_cols = df_train.select_dtypes(include=['number']).columns
categorical = list(categorical_cols)
numerical = list(numerical_cols)

In [21]:
from sklearn.metrics import mutual_info_score

In [22]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.y)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
df_mi['MI'] = df_mi['MI'].round(2)


display(df_mi)

,MI
poutcome,0.03
month,0.02
contact,0.01
housing,0.01
job,0.01
education,0.00
marital,0.00


In [23]:
from sklearn.feature_extraction import DictVectorizer

In [24]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [25]:
train_dict[0]

{'job': 'technician',
 'marital': 'single',
 'education': 'tertiary',
 'housing': 'yes',
 'contact': 'cellular',
 'month': 'aug',
 'poutcome': 'unknown',
 'age': 32,
 'balance': 1100,
 'day': 11,
 'duration': 67,
 'campaign': 1,
 'pdays': -1,
 'previous': 0}

In [26]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dict)

In [27]:
X_train.shape

(27126, 47)

In [28]:
dv.get_feature_names_out()

array(['age', 'balance', 'campaign', 'contact=cellular',
       'contact=telephone', 'contact=unknown', 'day', 'duration',
       'education=primary', 'education=secondary', 'education=tertiary',
       'education=unknown', 'housing=no', 'housing=yes', 'job=admin.',
       'job=blue-collar', 'job=entrepreneur', 'job=housemaid',
       'job=management', 'job=retired', 'job=self-employed',
       'job=services', 'job=student', 'job=technician', 'job=unemployed',
       'job=unknown', 'marital=divorced', 'marital=married',
       'marital=single', 'month=apr', 'month=aug', 'month=dec',
       'month=feb', 'month=jan', 'month=jul', 'month=jun', 'month=mar',
       'month=may', 'month=nov', 'month=oct', 'month=sep', 'pdays',
       'poutcome=failure', 'poutcome=other', 'poutcome=success',
       'poutcome=unknown', 'previous'], dtype=object)

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [31]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [32]:
model.predict_proba(X_val)


array([[0.98759451, 0.01240549],
       [0.98982363, 0.01017637],
       [0.84484044, 0.15515956],
       ...,
       [0.94323596, 0.05676404],
       [0.99091088, 0.00908912],
       [0.71500464, 0.28499536]])

In [33]:
y_pred = model.predict_proba(X_val)[:, 1]

In [34]:
class1 = y_pred >=  0.5

In [35]:
accutacy_all_features = (y_val == class1).mean()
accutacy_all_features

0.9009068790090687

Question 5

Let's find the least useful feature using the feature elimination technique.
Train a model with all these features (using the same parameters as in Q4).
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?


age

balance

marital

previous


Note: The difference doesn't have to be positive.

In [36]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
exclude_features = ["age","balance","marital","previous"]
for feature in exclude_features:
  selected_features = list(set(numerical + categorical) - set([feature]))
  #print(feature ,"\n" , selected_features )
  train_dict = df_train[selected_features].to_dict(orient='records')
  X_train = dv.fit_transform(train_dict)
  model.fit(X_train, y_train)
  val_dict = df_val[selected_features].to_dict(orient='records')
  X_val = dv.transform(val_dict)
  y_pred = model.predict_proba(X_val)[:, 1]
  class1 = y_pred >=  0.5
  accuracy = (y_val == class1).mean()
  print(feature,accutacy_all_features - accuracy)



age -0.00044238000442387015
balance -0.0001105950011059953
marital 0.0
previous 0.0


In [37]:
hyp_par = [0.01, 0.1, 1, 10, 100]
train_dict = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dict)
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

for c in hyp_par:
  model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict_proba(X_val)[:, 1]
  class1 = y_pred >=  0.5
  accuracy = (y_val == class1).mean()
  print(c , accuracy)

0.01 0.8979208139792081
0.1 0.9007962840079629
1 0.9009068790090687
10 0.9009068790090687
100 0.9006856890068569
